# Test notebook - modularity functions with HNX2

We compare functions in the modularity module of HNX with new proposed versions;

This is to be pulled into an upcoming version of HNX, currently in: https://github.com/ftheberge/HyperNetX/tree/modularity



In [1]:
import pandas as pd
import numpy as np
import igraph as ig
import partition_igraph
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod ## new as of version 1.2
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from functools import reduce
import itertools
from scipy.special import comb
import warnings
warnings.simplefilter('ignore')

hnx.__version__

'2.0.4'

In [2]:
Datadir = "../Data/"

# Updates to HNX2.0 modularity

### Unchanged functions:

- dict2part(D)
- part2dict(A)
- linear(d, c)
- majority(d, c)
- strict(d, c)
- two_section(HG)
- kumar(HG, delta=0.01)

### No longer required

- precompute_attributes(H)
- _compute_partition_probas(HG, A)
- _degree_tax(HG, Pr, wdc)
- _edge_contribution(HG, A, wdc)
- _delta_ec(HG, P, v, a, b, wdc)
- _bin_ppmf(d, c, p)
- _delta_dt(HG, P, v, a, b, wdc)

### New version (temporarily renamed with prefix “new_”)

- modularity(HG, A, wdc=linear)
- last_step(HG, L, wdc=linear, delta=0.01)

### New functions

- _last_step_unweighted
- _last_step_weighted


# New or updated functions

In [3]:
## New proposed implementation for modularity

from collections import Counter
from scipy.stats import binom 

def new_modularity(H, A, wdc=hmod.linear):

    ## all same edge weights?
    uniq = (len(Counter(H.edges.properties['weight']))==1)
    
    ## Edge Contribution
    H_id = H.incidence_dict
    d = hmod.part2dict(A)
    L = [ [d[i] for i in H_id[x]] for x in H_id ]

    ## all same weight
    if uniq:
        _ctr = Counter([ (Counter(l).most_common(1)[0][1],len(l)) for l in L])
        EC = sum([wdc(k[1],k[0])*_ctr[k] for k in _ctr.keys() if k[0] > k[1]/2])
    else:
        _keys = [ (Counter(l).most_common(1)[0][1],len(l)) for l in L]
        _vals = list(H.edge_props['weight']) ## Thanks Brenda!!
        _df = pd.DataFrame(zip(_keys,_vals), columns=['key','val'])
        _df = _df.groupby(by='key').sum()
        EC = sum([ wdc(k[1],k[0])*v[0] for (k,v) in _df.iterrows() if k[0]>k[1]/2 ])
        
    ## Degree Tax
    if uniq:        
        VolA = [sum([H.degree(i) for i in k]) for k in A]
        Ctr = Counter([H.size(i) for i in H.edges])

    else:
        ## this is the bottleneck
        VolA = np.repeat(0,1+np.max(list(d.values())))
        m = np.max([H.size(i) for i in H.edges])
        Ctr = np.repeat(0,1+m)
        S = 0
        for e in H.edges:
            w = H.edges[e].weight
            Ctr[H.size(e)] += w  
            S += w
            for v in H.edges[e]:
                VolA[d[v]] += w 
                
    VolV = np.sum(VolA)
    VolA = [x/VolV for x in VolA]
    DT = 0
    
    if uniq:        
        for d in Ctr.keys():
            Cnt = Ctr[d]
            for c in np.arange(int(np.floor(d/2+1)),d+1):
                for Vol in VolA:
                    DT += (Cnt*wdc(d,c)*binom.pmf(c,d,Vol))
        return (EC-DT)/H.number_of_edges()
    else:
        for d in range(len(Ctr)):
            Cnt = Ctr[d]
            for c in np.arange(int(np.floor(d/2+1)),d+1):
                for Vol in VolA:
                    DT += (Cnt*wdc(d,c)*binom.pmf(c,d,Vol))
        return (EC-DT)/S

In [4]:
## THIS ASSUMES UNWEIGHTED H 
def _last_step_unweighted(H, A, wdc, delta=0.01):

    qH = new_modularity(H,A,wdc=wdc)
    print('initial qH:',qH)

    ## initialize
    ctr_sizes = Counter([H.size(i) for i in H.edges])
    VolA = [sum([H.degree(i) for i in k]) for k in A]
    VolV = np.sum(VolA)
    dct_A = hmod.part2dict(A)

    while(True):
        
        n_moves = 0
        for v in list(np.random.permutation(list(H.nodes))):

            dct_A_v = dct_A[v]
            H_id = [H.incidence_dict[x] for x in H.nodes[v].memberships]
            L = [ [dct_A[i] for i in x] for x in H_id ]
            deg_v = H.degree(v)

            ## assume unweighted - EC portion before
            _ctr = Counter([ (Counter(l).most_common(1)[0][1],len(l)) for l in L])
            ec = sum([wdc(k[1],k[0])*_ctr[k] for k in _ctr.keys() if k[0] > k[1]/2])

            ## DT portion before
            dt = 0
            for d in ctr_sizes.keys():
                Cnt = ctr_sizes[d]
                for c in np.arange(int(np.floor(d/2+1)),d+1):
                    dt += (Cnt*wdc(d,c)*binom.pmf(c,d,VolA[dct_A_v]/VolV)) 
            
            ## move it?
            best = dct_A_v
            best_del_q = 0
            best_dt = 0
            for m in set([i for x in L for i in x])-{dct_A_v}:
                dct_A[v] = m
                L = [ [dct_A[i] for i in x] for x in H_id ]
                ## assume unweighted - EC
                _ctr = Counter([ (Counter(l).most_common(1)[0][1],len(l)) for l in L])
                ecp = sum([wdc(k[1],k[0])*_ctr[k] for k in _ctr.keys() if k[0] > k[1]/2])
                ## DT
                del_dt = -dt
                for d in ctr_sizes.keys():
                    Cnt = ctr_sizes[d]
                    for c in np.arange(int(np.floor(d/2+1)),d+1):
                        del_dt -= (Cnt*wdc(d,c)*binom.pmf(c,d,VolA[m]/VolV))
                        del_dt += (Cnt*wdc(d,c)*binom.pmf(c,d,(VolA[m]+deg_v)/VolV))
                        del_dt += (Cnt*wdc(d,c)*binom.pmf(c,d,(VolA[dct_A_v]-deg_v)/VolV)) 
                del_q = ecp-ec-del_dt
                if del_q > best_del_q:
                    best_del_q = del_q
                    best = m
                    best_dt = del_dt
            if best_del_q > 0.1: ## this avoids some numerical precision issues
                n_moves += 1
                dct_A[v] = best
                VolA[m] += deg_v
                VolA[dct_A_v] -= deg_v
                VolV = np.sum(VolA)
            else:
                dct_A[v] = dct_A_v
        new_qH = new_modularity(H, hmod.dict2part(dct_A), wdc=wdc)    
        print(n_moves,'moves, new qH:',new_qH)
        if (new_qH-qH) < delta:
            break
        else:
            qH = new_qH
    return hmod.dict2part(dct_A)

In [5]:
## THIS ASSUMES WEIGHTED H
def _last_step_weighted(H, A, wdc, delta=0.01):

    qH = new_modularity(H,A,wdc=wdc)
    print('initial qH:',qH)
    d = hmod.part2dict(A)

    ## initialize
    ## this is the bottleneck
    VolA = np.repeat(0,1+np.max(list(d.values())))
    m = np.max([H.size(i) for i in H.edges])
    ctr_sizes = np.repeat(0,1+m)
    S = 0
    for e in H.edges:
        w = H.edges[e].weight
        ctr_sizes[H.size(e)] += w  
        S += w
        for v in H.edges[e]:
            VolA[d[v]] += w 
    VolV = np.sum(VolA)
    dct_A = hmod.part2dict(A)

    ## loop
    while(True):
        n_moves = 0
        for v in list(np.random.permutation(list(H.nodes))):

            dct_A_v = dct_A[v]
            H_id = [H.incidence_dict[x] for x in H.nodes[v].memberships]
            L = [ [dct_A[i] for i in x] for x in H_id ]

            ## ec portion before move
            _keys = [ (Counter(l).most_common(1)[0][1],len(l)) for l in L]
            _vals = [H.edge_props['weight'][x] for x in H.nodes[v].memberships]
            _df = pd.DataFrame(zip(_keys,_vals), columns=['key','val'])
            _df = _df.groupby(by='key').sum()
            ec = sum([ wdc(k[1],k[0])*val[0] for (k,val) in _df.iterrows() if k[0]>k[1]/2 ])
            str_v = np.sum(_vals) ## weighted degree

            ## DT portion before move
            dt = 0
            for d in range(len(ctr_sizes)):
                Cnt = ctr_sizes[d]
                for c in np.arange(int(np.floor(d/2+1)),d+1):
                    dt += (Cnt*wdc(d,c)*binom.pmf(c,d,VolA[dct_A_v]/VolV)) 


            ## move it?
            best = dct_A_v
            best_del_q = 0
            best_dt = 0
            for m in set([i for x in L for i in x])-{dct_A_v}:
                dct_A[v] = m
                L = [ [dct_A[i] for i in x] for x in H_id ]
                ## EC
                _keys = [ (Counter(l).most_common(1)[0][1],len(l)) for l in L]
                _vals = [H.edge_props['weight'][x] for x in H.nodes[v].memberships]
                _df = pd.DataFrame(zip(_keys,_vals), columns=['key','val'])
                _df = _df.groupby(by='key').sum()
                ecp = sum([ wdc(k[1],k[0])*val[0] for (k,val) in _df.iterrows() if k[0]>k[1]/2 ])    


                ## DT
                del_dt = -dt
                for d in range(len(ctr_sizes)):
                    Cnt = ctr_sizes[d]
                    for c in np.arange(int(np.floor(d/2+1)),d+1):
                        del_dt -= (Cnt*wdc(d,c)*binom.pmf(c,d,VolA[m]/VolV))
                        del_dt += (Cnt*wdc(d,c)*binom.pmf(c,d,(VolA[m]+str_v)/VolV))
                        del_dt += (Cnt*wdc(d,c)*binom.pmf(c,d,(VolA[dct_A_v]-str_v)/VolV))
                del_q = ecp-ec-del_dt
                if del_q > best_del_q:
                    best_del_q = del_q
                    best = m
                    best_dt = del_dt

            if best_del_q > 0.1: ## this avoids some precision issues
                n_moves += 1
                dct_A[v] = best
                VolA[m] += str_v
                VolA[dct_A_v] -= str_v
                VolV = np.sum(VolA)
            else:
                dct_A[v] = dct_A_v

        new_qH = new_modularity(H, hmod.dict2part(dct_A), wdc=wdc)
        print(n_moves,'moves, new qH:',new_qH)
        if (new_qH-qH) < delta:
            break
        else:
            qH = new_qH
    return hmod.dict2part(dct_A)

In [6]:
def new_last_step(H, A, wdc=hmod.linear, delta=0.01):
    
    ## all same edge weights?
    uniq = (len(Counter(H.edges.properties['weight']))==1)

    if uniq:
        nls = _last_step_unweighted(H, A, wdc=wdc, delta=delta)
    else:
        nls = _last_step_weighted(H, A, wdc=wdc, delta=delta)
    return nls

# Experiment with h-ABCD hypergraphs

We generated 4 h-ABCD hypergraphs with parameters:

* -n 1000 -d 2.5,5,50 -c 1.5,50,200 -x 0.5 -q 0.0,0.4,0.3,0.2,0.1 -w :**linear** -s 1234 -o linear_1000
* same as above with **strict**, **majority**
* -n 1000 -d 2.5,5,50 -c 1.5,50,200 -x 0.5 -q 0.0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1 -w :linear -s 1234 -o linear_large_edges_1000
    
    

In [7]:
file_vertex_labels = Datadir+'linear_1000_assign.txt'
file_hyperedges = Datadir+'linear_1000_he.txt'
with open(file_hyperedges, 'r') as file:
    # Read all the lines of the file into a list
    lines = file.readlines()
hyperedges = [[y for y in x.replace('\n','').split(',')] for x in lines]

with open(file_vertex_labels, 'r') as file:
    # Read all the lines of the file into a list
    vertex_labels = np.array([int(y) for y in file.read().splitlines()])

H = hnx.Hypergraph(hyperedges)
H.shape    

(1000, 3385)

In [8]:
%%time
## Cluster the 2-section graph (with Louvain)
G = hmod.two_section(H)
G.vs['louvain'] = G.community_multilevel(weights='weight').membership
ML = hmod.dict2part({v['name']:v['louvain'] for v in G.vs})


CPU times: user 176 ms, sys: 226 µs, total: 176 ms
Wall time: 176 ms


In [9]:
%%time
## pre-compute required quantities for modularity and clustering with current HNX functions
H = hmod.precompute_attributes(H)


CPU times: user 13.5 s, sys: 73.9 ms, total: 13.6 s
Wall time: 13.6 s


In [10]:
%%time
## Compute qH with current HNX function
print('q-linear:',hmod.modularity(H, ML, wdc=hmod.linear))
print('q-majority:',hmod.modularity(H, ML, wdc=hmod.majority))
print('q-strict:',hmod.modularity(H, ML, wdc=hmod.strict))


q-linear: 0.37731947885116807
q-majority: 0.38903123340397894
q-strict: 0.347003775016151
CPU times: user 4.36 s, sys: 24.6 ms, total: 4.38 s
Wall time: 4.38 s


In [11]:
%%time
## Compute qH with new function
print('q-linear:',new_modularity(H, ML, wdc=hmod.linear))
print('q-majority:',new_modularity(H, ML, wdc=hmod.majority))
print('q-strict:',new_modularity(H, ML, wdc=hmod.strict))

q-linear: 0.37731947885116546
q-majority: 0.3890312334039788
q-strict: 0.34700377501615104
CPU times: user 114 ms, sys: 0 ns, total: 114 ms
Wall time: 113 ms


In [12]:
%%time
KU = hmod.kumar(H)


CPU times: user 7.7 s, sys: 18.6 ms, total: 7.72 s
Wall time: 7.72 s


In [13]:
%%time
## Compute qH with current HNX function
print('q-linear:',hmod.modularity(H, KU, wdc=hmod.linear))
print('q-majority:',hmod.modularity(H, KU, wdc=hmod.majority))
print('q-strict:',hmod.modularity(H, KU, wdc=hmod.strict))


q-linear: 0.36809928524991276
q-majority: 0.3812803723167444
q-strict: 0.33497415987329315
CPU times: user 4.31 s, sys: 24.4 ms, total: 4.34 s
Wall time: 4.34 s


In [14]:
%%time
## Compute qH with new function
print('q-linear:',new_modularity(H, KU, wdc=hmod.linear))
print('q-majority:',new_modularity(H, KU, wdc=hmod.majority))
print('q-strict:',new_modularity(H, KU, wdc=hmod.strict))

q-linear: 0.36809928524991065
q-majority: 0.38128037231674433
q-strict: 0.3349741598732931
CPU times: user 46.4 ms, sys: 3.49 ms, total: 49.9 ms
Wall time: 48.6 ms


In [15]:
%%time
KU_nls = new_last_step(H, KU, wdc=hmod.linear)

initial qH: 0.36809928524991065
100 moves, new qH: 0.38514228504186865
38 moves, new qH: 0.38932093928072026
CPU times: user 22.1 s, sys: 132 ms, total: 22.2 s
Wall time: 22.2 s


In [16]:
%%time
## Compute qH with current HNX function
print('q-linear:',hmod.modularity(H, KU_nls, wdc=hmod.linear))
print('q-majority:',hmod.modularity(H, KU_nls, wdc=hmod.majority))
print('q-strict:',hmod.modularity(H, KU_nls, wdc=hmod.strict))

q-linear: 0.389320939280723
q-majority: 0.41813771866779315
q-strict: 0.32122263643404636
CPU times: user 4.37 s, sys: 11.9 ms, total: 4.38 s
Wall time: 4.38 s


In [17]:
%%time
## Compute qH with new function
print('q-linear:',new_modularity(H, KU_nls, wdc=hmod.linear))
print('q-majority:',new_modularity(H, KU_nls, wdc=hmod.majority))
print('q-strict:',new_modularity(H, KU_nls, wdc=hmod.strict))

q-linear: 0.38932093928072026
q-majority: 0.41813771866779315
q-strict: 0.32122263643404636
CPU times: user 48.3 ms, sys: 41 µs, total: 48.4 ms
Wall time: 47 ms
